Recipe URL Generator

This program gives recipe urls from a website called Minimalist Baker. It asks the user what ingredient one have at home. With the given answer it stores the urls until the user enters in 'end'. When the user finishes with their recipe searches it gives out all the recipe urls. #the program takes a bit of time to load#
The result is that based on the ingredients they have at home they can immediately find a suitable recipe. It can help to reduce time spent on searching for recipes.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import HTML
from IPython.display import clear_output
from bs4 import BeautifulSoup #for the webscraping I needed to import BeautifulSoup and requests
import pandas as pd  
import requests
#import time #with the early code I used all of these but at the end I could finish the code without them
#import re
#import numpy as np

#since the each recipe has different urls, and the recipe index has 78 pages, I had to create a function to scrape every url into a list.
url_base = "https://minimalistbaker.com/recipe-index/?fwp_paged="
page = requests.get(url_base)
soup = BeautifulSoup(page.text,"html.parser")


recipes = []
for i in range(1,79):
    
    url = url_base + str(i)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text,"html.parser")
    recipe_urls = pd.Series([a.get("href") for a in soup.find_all("a")])
    recipe_urls = recipe_urls[(recipe_urls.str.contains("work")== False) &
                          (recipe_urls.str.contains("recipe-index")==False)&
                          ((recipe_urls.str.contains("easy")==True)|        
                          (recipe_urls.str.contains("vegan")==True)| 
                          (recipe_urls.str.contains("gluten-free")==True)|
                          (recipe_urls.str.contains("with")==True))].unique() #this is how I selected, which url was the ones I need
    recipes.extend(recipe_urls) #recipes is the list of recipe urls

In [2]:
i_list = recipes 
#I created two dictionaries, which contain the ingredient and url pairs.
urlDict = {} #the key is the url and the values are the ingredients - for every url it gives what ingredient it contains
ingDict = {} #the key is the ingredient - for every ingredient it pairs it with its url

for url in i_list:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text,"html.parser")

    ingredients = []
    lis = []
    

    for ultag in soup.find_all('ul', {'class': 'wprm-recipe-ingredients'}): #here I had to scrape the ingredients
        for litag in ultag.find_all('li'):
            name = litag.find('span', {'class': 'wprm-recipe-ingredient-name'}) 
            #print(name.text.lower())
            ingredient = name.text.lower() #so no difference between capital or lower case letters - working with lower case
            lis.append(ingredient)
            if ingredient in ingDict.keys():
                if url not in ingDict[ingredient]:
                    ingDict[ingredient].append(url)        
            else:
                ingDict[ingredient]=[url]

    urlDict[url]=lis    
    #print(ingDict)

In [ ]:
input_loop = True
guesses = [] #list for the users inputs
while input_loop == True: #I set up a loop, which will allow the user to search for urls till they enter end
    x = input('What ingredients do you have at home? Add one, you would like to see recipes with! You can enter one ingredient at a time, and you will get the recipe urls. You can search for several ingredients, but only type in one at a time. The program stores your searches and when you have finished your search, please enter end and it will show your results! Please use lower-case letters.')
    clear_output()
    if x == 'end': #if the user enters end the loop will stop and it will give out the answers
        input_loop = False
    else:
        guesses.append(x) #Here the input will be out into the guesses list 
for item in guesses:   
    if ingDict.get(item) == None:
        print('You used upper-case or there are no recipes that contain your ingredient, please try again!') #important to use lower-case, because in line 19 I changed every ingredient to lower-case so each will be taken into account
    else:
        print('These are the recipe urls you could make with:', item,  ingDict.get(item), ) #gives out the answers


In [3]:
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ingDict.items() ]))
df # a dataframe with ingredients as columns, so the multiple recipe urls, which contain the same ingredient are easier to see :) 

,coconut oil,maple syrup,coconut sugar,blanched slivered almonds,sea salt,"cauliflower, cut into small florets",olive oil,finely chopped onion,"garlic, minced",cornstarch or arrowroot starch,...,all-purpose or pastry flour*,cream cake cones,vegan honey such as bee free,soymilk,maple syrup or agave*,quality espresso or strong coffee,carrot,whole wheat spaghetti,tomato sauce,sun dried tomato spread
0,https://minimalistbaker.com/easy-almond-brittl...,https://minimalistbaker.com/easy-almond-brittl...,https://minimalistbaker.com/easy-almond-brittl...,https://minimalistbaker.com/easy-almond-brittl...,https://minimalistbaker.com/easy-almond-brittl...,https://minimalistbaker.com/creamy-vegan-cauli...,https://minimalistbaker.com/creamy-vegan-cauli...,https://minimalistbaker.com/creamy-vegan-cauli...,https://minimalistbaker.com/creamy-vegan-cauli...,https://minimalistbaker.com/creamy-vegan-cauli...,...,https://minimalistbaker.com/vegan-peach-cupcak...,https://minimalistbaker.com/vegan-peach-cupcak...,https://minimalistbaker.com/vegan-peach-cupcak...,https://minimalistbaker.com/espresso-glazed-ve...,https://minimalistbaker.com/espresso-glazed-ve...,https://minimalistbaker.com/espresso-glazed-ve...,https://minimalistbaker.com/three-color-pasta-...,https://minimalistbaker.com/three-color-pasta-...,https://minimalistbaker.com/three-color-pasta-...,https://minimalistbaker.com/three-color-pasta-...
1,https://minimalistbaker.com/vegan-gluten-free-...,https://minimalistbaker.com/easy-orange-cranbe...,https://minimalistbaker.com/1-bowl-chocolate-p...,NaN,https://minimalistbaker.com/easy-roasted-potat...,NaN,https://minimalistbaker.com/easy-beet-fritters...,NaN,https://minimalistbaker.com/creamy-vegan-mushr...,https://minimalistbaker.com/easy-vegan-flan-no...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://minimalistbaker.com/easy-homemade-pean...,https://minimalistbaker.com/pecan-pie-bars-veg...,https://minimalistbaker.com/pumpkin-swirl-chee...,NaN,https://minimalistbaker.com/1-bowl-vegan-glute...,NaN,https://minimalistbaker.com/miso-pumpkin-risot...,NaN,https://minimalistbaker.com/miso-pumpkin-risot...,https://minimalistbaker.com/mini-vegan-pumpkin...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://minimalistbaker.com/apple-butter-chees...,https://minimalistbaker.com/easy-baked-vegan-c...,https://minimalistbaker.com/easy-apple-coffee-...,NaN,https://minimalistbaker.com/1-pot-smoky-lentil...,NaN,https://minimalistbaker.com/wild-rice-salad-wi...,NaN,https://minimalistbaker.com/creamy-potato-leek...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://minimalistbaker.com/vegan-coffee-cake-...,https://minimalistbaker.com/pumpkin-swirl-chee...,https://minimalistbaker.com/gluten-free-pancak...,NaN,https://minimalistbaker.com/creamy-vegan-milk-...,NaN,https://minimalistbaker.com/creamy-potato-leek...,NaN,https://minimalistbaker.com/vegan-chorizo-saus...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
